<a href="https://colab.research.google.com/github/jammy-bot/keras-to-pytorch/blob/features/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

from urllib.request import urlretrieve

In [2]:
# reading the dataset
iris = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
urlretrieve(iris)
df = pd.read_csv(iris, sep=',')

# adding column names to the dataframe
df.columns = ["sepal_length", "sepal_width", 
              "petal_length", "petal_width", 
              "species"]


In [3]:
# viewing a random sample of rows from the dataframe
df.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
96,6.2,2.9,4.3,1.3,Iris-versicolor
110,6.4,2.7,5.3,1.9,Iris-virginica
26,5.2,3.5,1.5,0.2,Iris-setosa
141,5.8,2.7,5.1,1.9,Iris-virginica
61,6.0,2.2,4.0,1.0,Iris-versicolor


In [4]:
# converting to numpy
x = df[["sepal_length", "sepal_width", 
        "petal_length", "petal_width"]].values

le = preprocessing.LabelEncoder()
y = le.fit_transform(df["species"])
species = le.classes_

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  149 non-null    float64
 1   sepal_width   149 non-null    float64
 2   petal_length  149 non-null    float64
 3   petal_width   149 non-null    float64
 4   species       149 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [6]:
# splitting into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state = 42)

# NN

In [7]:
# Build neural network
# model = Sequential()
# model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
# model.add(Dense(25, activation='relu')) # Hidden 2
# model.add(Dense(y.shape[1],activation='softmax')) # Output

# model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.fit(x_train,y_train,verbose=2,epochs=100)

# import pytorch library and modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [8]:
class Net(nn.Module):
    def __init__(self, in_count, output_count):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_count, 50) # first hidden layer
        self.fc2 = nn.Linear(50, 25) # 50 form hidden1 to 25 in hidden2
        self.fc3 = nn.Linear(25, output_count)
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return self.LogSoftmax(x)

In [9]:
# converting data to pytorch tensors
x_train= Variable(torch.Tensor(x_train).float())
x_test= Variable(torch.Tensor(x_test).float())

# converting class indexes
y_train= Variable(torch.LongTensor(y_train))
y_test= Variable(torch.LongTensor(y_test))

In [10]:
# instantiating the model
model = Net(x.shape[1], len(species)) # number of columns, rows
criterion = nn.CrossEntropyLoss() # declaring the loss function
optimizer = torch.optim.Adam(model.parameters(),  # parameters are the weights
                             lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    out = model(x_train)
    loss = criterion(out, y_train)
    loss.backward() # backward propagation
    optimizer.step() # applying the gradients
    print(f"Epoch: {epoch+1}, loss: {loss.item()}")

Epoch: 1, loss: 1.4045157432556152
Epoch: 2, loss: 1.0766183137893677
Epoch: 3, loss: 1.0201165676116943
Epoch: 4, loss: 0.980073094367981
Epoch: 5, loss: 0.9165154695510864
Epoch: 6, loss: 0.8034027814865112
Epoch: 7, loss: 0.7558691501617432
Epoch: 8, loss: 0.7355009913444519
Epoch: 9, loss: 0.6813400387763977
Epoch: 10, loss: 0.613172173500061
Epoch: 11, loss: 0.5616206526756287
Epoch: 12, loss: 0.5187031030654907
Epoch: 13, loss: 0.4823765456676483
Epoch: 14, loss: 0.46467944979667664
Epoch: 15, loss: 0.44847372174263
Epoch: 16, loss: 0.4167915880680084
Epoch: 17, loss: 0.38802486658096313
Epoch: 18, loss: 0.37125128507614136
Epoch: 19, loss: 0.3462538421154022
Epoch: 20, loss: 0.31348881125450134
Epoch: 21, loss: 0.2895593047142029
Epoch: 22, loss: 0.2662048041820526
Epoch: 23, loss: 0.2374628633260727
Epoch: 24, loss: 0.21806937456130981
Epoch: 25, loss: 0.19758391380310059
Epoch: 26, loss: 0.1732998788356781
Epoch: 27, loss: 0.1594133973121643
Epoch: 28, loss: 0.1415640413761139

# Evaluate Accuracy

In [11]:
from sklearn.metrics import accuracy_score

pred = model(x_test)
# dunder to ignore the first parameter
_ , predict_classes = torch.max(pred,1) # np.argmax(pred,axis=1)
# expected_classes = np.argmax(y_test,axis=1)

correct = accuracy_score(y_test, predict_classes) # (expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.9473684210526315
